# Phase 3: SQL Analysis

**Objective:**  
Perform SQL analysis on cleaned datasets to answer key business questions, including revenue trends, customer behavior, and churn patterns.

**Datasets Used:**  
1. Retail Transactions (`retail`)  
2. Customer Support Tickets (`support`)  
3. Telecom Customer Data (`telco`)  

**Tools Used:**  
- SQLite (local database)  
- Python (pandas, sqlite3)
- Jupyter Notebook

### 1️⃣ Import libraries and load cleaned data CSV

In [1]:
import pandas as pd
import sqlite3
import os

# Create folder for SQL outputs
os.makedirs("sql_outputs", exist_ok=True)

# Load cleaned datasets from the data folder
retail = pd.read_csv("data/retail_cleaned.csv")   
support = pd.read_csv("data/support_cleaned.csv")
telco = pd.read_csv("data/telco_cleaned.csv")

# Check first few rows
retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalAmount
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


### 2️⃣ Create SQLite Database and load dataframes into SQLite

In [2]:
# Create SQLite in-memory database
conn = sqlite3.connect(":memory:")

# Load dataframes into SQLite
retail.to_sql("retail", conn, index=False, if_exists="replace")
support.to_sql("support", conn, index=False, if_exists="replace")
telco.to_sql("telco", conn, index=False, if_exists="replace")

7032

#### QUERY 1: TOTAL REVENUE BY COUNTRY

In [3]:
query1 = """
SELECT Country,
       ROUND(SUM(TotalAmount), 2) AS TotalRevenue
FROM retail
WHERE Country IS NOT NULL
GROUP BY Country
ORDER BY TotalRevenue DESC;
"""
revenue_country = pd.read_sql(query1, conn)
revenue_country.to_csv("sql_outputs/revenue_country.csv", index=False)
print("Query 1 executed and saved.")

Query 1 executed and saved.


#### QUERY 2: TOP 10 CUSTOMERS BY TOTAL SPENT

In [4]:
query2 = """
SELECT "Customer ID",
       ROUND(SUM(TotalAmount), 2) AS TotalSpent,
       COUNT(*) AS TotalOrders
FROM retail
GROUP BY "Customer ID"
ORDER BY TotalSpent DESC
LIMIT 10;
"""
top_customers = pd.read_sql(query2, conn)
top_customers.to_csv("sql_outputs/top_customers.csv", index=False)
print("Query 2 executed and saved.")

Query 2 executed and saved.


#### QUERY 3: MONTHLY REVENUE TRENDS

In [5]:
query3 = """
SELECT STRFTIME('%Y-%m', InvoiceDate) AS YearMonth,
       ROUND(SUM(TotalAmount), 2) AS MonthlyRevenue
FROM retail
GROUP BY YearMonth
ORDER BY YearMonth;
"""
monthly_revenue = pd.read_sql(query3, conn)
monthly_revenue.to_csv("sql_outputs/monthly_revenue.csv", index=False)
print("Query 3 executed and saved.")

Query 3 executed and saved.


#### QUERY 4: CUSTOMER WITH MOST SUPPORT TICKETS

In [6]:
query4 = """
SELECT s."Customer Name",
       COUNT(s."Ticket ID") AS TotalTickets
FROM support s
GROUP BY s."Customer Name"
ORDER BY TotalTickets DESC
LIMIT 10;
"""
top_support_customers = pd.read_sql(query4, conn)
top_support_customers.to_csv("sql_outputs/top_support_customers.csv", index=False)
print("Query 4 executed and saved.")

Query 4 executed and saved.


#### QUERY 5: RETAIL-TOP PRODUCTS BY REVENUE

In [7]:
query5 = """
SELECT Description AS ProductName,
       SUM(TotalAmount) AS RevenueGenerated,
       COUNT(*) AS UnitsSold
FROM retail
GROUP BY ProductName
ORDER BY RevenueGenerated DESC
LIMIT 10;
"""
top_products= pd.read_sql(query5, conn)
top_products.to_csv("sql_outputs/top_products.csv", index=False)
print("Query 5 executed and saved.")

Query 5 executed and saved.


#### QUERY 6: TOP 5 TICKET TYPES

In [8]:
query6 = """
SELECT [Ticket Type] AS TicketType,
       COUNT(*) AS TotalTickets
FROM support
GROUP BY [Ticket Type]
ORDER BY TotalTickets DESC
LIMIT 5;
"""
ticket_type = pd.read_sql(query6, conn)
ticket_type.to_csv("sql_outputs/ticket_type.csv", index=False)
print("Query 6 executed and saved.")

Query 6 executed and saved.


#### QUERY 7: AVERAGE RESOLUTION TIME BY TICKET TYPE

In [9]:
query7 = """
SELECT [Ticket Type] AS TicketType,
       ROUND(AVG([Time to Resolution]),2) AS AvgResolutionHours,
       COUNT(*) AS TotalTickets
FROM support
GROUP BY [Ticket Type]
ORDER BY AvgResolutionHours DESC;
"""
resolution_time = pd.read_sql(query7, conn)
resolution_time.to_csv("sql_outputs/resolution_time.csv", index=False)
print("Query 7 executed and saved.")

Query 7 executed and saved.


#### QUERY 8: CUSTOMER SATISFACTION BY TICKET PRIORITY

In [10]:
query8 = """
SELECT [Ticket Priority] AS Priority,
       ROUND(AVG([Customer Satisfaction Rating]),2) AS AvgSatisfaction,
       COUNT(*) AS TotalTickets
FROM support
GROUP BY [Ticket Priority]
ORDER BY AvgSatisfaction DESC;
"""
satisfaction_priority = pd.read_sql(query8, conn)
satisfaction_priority.to_csv("sql_outputs/satisfaction_priority.csv", index=False)
print("Query 8 executed and saved.")

Query 8 executed and saved.


#### QUERY 9: CHURN RATE BY CONTRACT TYPE

In [11]:
query9 = """
SELECT Contract,
       ROUND(AVG(Churn_flag)*100,2) AS ChurnRatePercentage,
       COUNT(*) AS TotalCustomers
FROM telco
GROUP BY Contract
ORDER BY ChurnRatePercentage DESC;
"""
churn_contract = pd.read_sql(query9, conn)
churn_contract.to_csv("sql_outputs/churn_contract.csv", index=False)
print("Query 9 executed and saved.")

Query 9 executed and saved.


In [12]:
#### QUERY 10: MONTHLY CHARGE BY CONTRACT TYPE

In [13]:
query10 = """
SELECT Contract,
       ROUND(AVG(MonthlyCharges),2) AS AvgMonthlyCharges,
       COUNT(*) AS TotalCustomers
FROM telco
GROUP BY Contract
ORDER BY AvgMonthlyCharges DESC;
"""
monthly_contract = pd.read_sql(query10, conn)
monthly_contract.to_csv("sql_outputs/monthly_contract.csv", index=False)

In [14]:
print("All SQL queries executed successfully. CSVs saved in 'sql_outputs' folder.")

All SQL queries executed successfully. CSVs saved in 'sql_outputs' folder.
